In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, TaskType, PeftConfig,PeftModel
import time

model_path = "models/cleaned_eos/gemma-2-9b-it-qlora-content1024/checkpoint-13638"
PeftConfig.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it",device_map="cuda")
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="cuda",torch_dtype=torch.bfloat16)

model = PeftModel.from_pretrained(model,
    model_path,
    is_trainable=True,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

# model = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-2-9b-it",
#     device_map="cuda",
#     torch_dtype=torch.bfloat16,
# )


/home/yangrumei/miniconda3/envs/gemma2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


In [2]:
print("模型各层的数据类型(dtype):")
for name, param in model.named_parameters():
    print(f"层名称: {name:<50} | dtype: {param.dtype}")

# 如果想要更简洁的统计，可以统计每种dtype的数量
print("\n数据类型统计:")
dtype_count = {}
for _, param in model.named_parameters():
    dtype = str(param.dtype)
    dtype_count[dtype] = dtype_count.get(dtype, 0) + 1

for dtype, count in dtype_count.items():
    print(f"{dtype}: {count} 层")

模型各层的数据类型(dtype):
层名称: base_model.model.model.embed_tokens.weight         | dtype: torch.bfloat16
层名称: base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight | dtype: torch.bfloat16
层名称: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight | dtype: torch.float32
层名称: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight | dtype: torch.float32
层名称: base_model.model.model.layers.0.self_attn.k_proj.weight | dtype: torch.bfloat16
层名称: base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight | dtype: torch.bfloat16
层名称: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight | dtype: torch.float32
层名称: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight | dtype: torch.float32
层名称: base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight | dtype: torch.bfloat16
层名称: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight | dtype: torch.float32
层名称: base_model.model.model.layers

In [3]:
input_text = "2+3等于多少？"
# 修改输入格式
input_text = f"<start_of_turn>user\n{input_text}<end_of_turn><eos>\n"
start_time = time.time()
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
tokenizer_time = time.time() - start_time
print(f"Tokenizer耗时: {tokenizer_time:.4f} 秒")
start_time = time.time()

outputs = model.generate(
    **input_ids,
    max_new_tokens=500,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    use_cache=True,
    do_sample=True,
    temperature=0.2,
    top_k=100,
    top_p=0.8,
)
generate_time = time.time() - start_time

print(f"Generate耗时: {generate_time:.4f} 秒")

# 解码输出时，我们可能需要清理一下输出，只保留模型回复部分
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

# 如果输出中包含了输入部分，可以尝试只提取模型回复部分
if "<start_of_turn>model\n" in generated_text:
    generated_text = generated_text.split("<start_of_turn>model\n")[1]
if "<end_of_turn>" in generated_text:
    generated_text = generated_text.split("<end_of_turn>")[0]
print("===")
print(generated_text)

Tokenizer耗时: 0.0013 秒
Generate耗时: 12.1926 秒
<bos><start_of_turn>user
2+3等于多少？<end_of_turn><eos>
<start_of_turn>model
我算了一下，10级大招的凝光伤害是：648*（1.5＋9）=11712这是我的面板。然后就是圣遗物加成了。（因为我没有好的岩伤杯子所以用的是攻击力百分比。）这套装备的话暴击率有接近百分之五十左右吧！爆伤也有三十多一点的样子！（没有截图_）。这样子的情况下一个e技能大概能打出两万多的输出啊！！！_(心情简单)这个数值还算是比较低的啦！！还有很多提升空间呢！！！！___下面是我带四风原典的面板：（没截到好图就先放一张自己打的单体怪的照片）（不是双火队哦！）可以看到一发重击大概是五千八百元？（不确定是不是这么高），但是如果加上元素反应那不得上七千了吧！！！！！__接下来是我的天赋等级和武器突破材料以及命座情况:最后还是希望大家能够多多支持一下呀!!!谢谢大家的观看!祝我们都能抽卡十连满命小保底不出金啊啊啊啊!!!!__(抱大腿)__拜托各位大佬们点个赞再走吧呜呜呜(๑•̀ㅂ•́)و✧٩( ᐛ )ง❀<end_of_turn><eos>
===
我算了一下，10级大招的凝光伤害是：648*（1.5＋9）=11712这是我的面板。然后就是圣遗物加成了。（因为我没有好的岩伤杯子所以用的是攻击力百分比。）这套装备的话暴击率有接近百分之五十左右吧！爆伤也有三十多一点的样子！（没有截图_）。这样子的情况下一个e技能大概能打出两万多的输出啊！！！_(心情简单)这个数值还算是比较低的啦！！还有很多提升空间呢！！！！___下面是我带四风原典的面板：（没截到好图就先放一张自己打的单体怪的照片）（不是双火队哦！）可以看到一发重击大概是五千八百元？（不确定是不是这么高），但是如果加上元素反应那不得上七千了吧！！！！！__接下来是我的天赋等级和武器突破材料以及命座情况:最后还是希望大家能够多多支持一下呀!!!谢谢大家的观看!祝我们都能抽卡十连满命小保底不出金啊啊啊啊!!!!__(抱大腿)__拜托各位大佬们点个赞再走吧呜呜呜(๑•̀ㅂ•́)و✧٩( ᐛ )ง❀
